<a href="https://colab.research.google.com/github/atharva0300/BE-8th-Semester/blob/main/nlp_mini_project/text_summarization_with_simple_transformers_t5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text summarization with Simple Transformers T5

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q sumeval==0.2.2
!pip install -q nlpaug==1.1.3
!pip install -q simpletransformers==0.60.9

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.2/394.2 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.7/206.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 61.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
! pip install -q nltk==3.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


## Import libraries <a class="anchor" id="Import-libraries"></a>

In [4]:
import gc
import random
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from wordcloud import WordCloud
from sklearn.model_selection import train_test_split

import nlpaug.augmenter.word as naw
from sumeval.metrics.rouge import RougeCalculator

import torch
from simpletransformers.t5 import T5Model, T5Args

print('Pytorch version: %s'  % torch.__version__)

Pytorch version: 2.2.1+cu121


In [5]:
warnings.simplefilter('ignore')
pd.set_option('display.max_colwidth', 10000)
cuda =  torch.cuda.is_available()

In [6]:
df = pd.read_csv('/content/drive/MyDrive/nlp_dataset/news_dataset_2.csv', encoding='ISO-8859-1').dropna().reset_index(drop=True)

## EDA <a class="anchor" id="EDA"></a>

In [7]:
display(df.head(1))

,content,summary
0,"indonesian : TEMPO.CO, Jakarta - Anggota Komisi VII DPR RI Rofik Hananto menyayangkan terjadinya insiden kebakaran yang disebabkan oleh bocornya depo Plumpang, Jakarta Utara, pada Jumat, 3 Maret 2023.""Turut berbela sungkawa terhadap masyarakat sipil yang terdampak, apalagi ini daerah padat penduduk. Pertamina harus tanggung jawab,"" kata dia dalam keterangan di Jakarta Sabtu, 4 Maret 2024.Rofik mengatakan Pertamina serta pihak terkait harus memastikan keselamatan dan keamanan warga yang tinggal di sekitar lokasi.Pipa BBM yang terbakar itu merupakan bagian dari Terminal Bahan Bakar Minyak (TBBM) Plumpang. TBBM Plumpang dinilai sebagai salah satu terminal BBM terpenting di Indonesia, sehingga Rofik juga menegaskan Pertamina harus memastikan pasokan bahan bakar minyak tetap aman meski ada insiden tersebut.""Plumpang menyuplai sekitar 20 persen kebutuhan BBM harian di Indonesia, atau sekitar 25 persen dari total kebutuhan SPBU Pertamina. Maka tindakan selanjutnya adalah bagaimana memastikan suplai BBM tidak terganggu,"" katanya.Rofik juga menyinggung kejadian kilang minyak milik Pertamina yang sebelumnya pernah terjadi kejadian serupa, seperti Kilang Balikpapan pada Maret 2022 lalu. Terhitung 2 tahun terakhir ini, kata dia, sudah 5 fasilitas migas milik Pertamina mengalami insiden kebakaran.""Menyayangkan terjadinya musibah kebakaran ini, apalagi insiden kebakaran bukan kali pertama di fasilitas migas, ini terus berulang. Pertamina harus benahi sistem pengamanan dan SOP yang ada,"" kata dia.Rofik mengharapkan adanya pembenahan menyeluruh terhadap sistem pengamanan serta SOP yang ada dalam lingkup Pertamina untuk mengamankan baik fasilitas migas, pekerja, maupun masyarakat sekitar. Menurutnya pembenahan diperlukan agar Pertamina tidak abai terhadap aspek pengamanan fasilitas migas.""Investigasi menyeluruh dan tuntas, serta meminta komitmen Pertamina memperbaiki sistem keamanan kilang minyak maupun depo BBM, seringnya kebakaran terjadi mengindikasikan Pertamina abai terhadap pengamanan kilang,"" ujarnya.Pilihan Editor:Profil Depo Pertamina Plumpang, Pemasok 20 Persen Pasokan BBM di Seluruh Indonesia","Anggota Komisi VII DPR RI Rofik Hananto menyayangkan terjadinya insiden kebakaran di Depo Plumpang, Jakarta Utara, pada Jumat, 3 Maret 2023. Rofik menegaskan Pertamina harus memastikan pasokan bahan bakar minyak tetap aman meski ada insiden tersebut. Depo Plumpang merupak"


In [8]:
df['content_length'] = [len(df['content'][i]) for i in range(len(df))]
df['summary_length'] = [len(df['summary'][i]) for i in range(len(df))]

In [9]:
print('df content length:\n', df['content_length'].describe())
print('df summary length:\n', df['summary_length'].describe())

df content length:
 count     12337.000000
mean       8575.640999
std       19707.364782
min          13.000000
25%        1702.000000
50%        2424.000000
75%        3963.000000
max      261837.000000
Name: content_length, dtype: float64
df summary length:
 count    12337.000000
mean       827.424414
std       1037.514210
min         89.000000
25%        394.000000
50%        489.000000
75%        601.000000
max      11394.000000
Name: summary_length, dtype: float64


In [10]:
display(df.head(1))

,content,summary,content_length,summary_length
0,"indonesian : TEMPO.CO, Jakarta - Anggota Komisi VII DPR RI Rofik Hananto menyayangkan terjadinya insiden kebakaran yang disebabkan oleh bocornya depo Plumpang, Jakarta Utara, pada Jumat, 3 Maret 2023.""Turut berbela sungkawa terhadap masyarakat sipil yang terdampak, apalagi ini daerah padat penduduk. Pertamina harus tanggung jawab,"" kata dia dalam keterangan di Jakarta Sabtu, 4 Maret 2024.Rofik mengatakan Pertamina serta pihak terkait harus memastikan keselamatan dan keamanan warga yang tinggal di sekitar lokasi.Pipa BBM yang terbakar itu merupakan bagian dari Terminal Bahan Bakar Minyak (TBBM) Plumpang. TBBM Plumpang dinilai sebagai salah satu terminal BBM terpenting di Indonesia, sehingga Rofik juga menegaskan Pertamina harus memastikan pasokan bahan bakar minyak tetap aman meski ada insiden tersebut.""Plumpang menyuplai sekitar 20 persen kebutuhan BBM harian di Indonesia, atau sekitar 25 persen dari total kebutuhan SPBU Pertamina. Maka tindakan selanjutnya adalah bagaimana memastikan suplai BBM tidak terganggu,"" katanya.Rofik juga menyinggung kejadian kilang minyak milik Pertamina yang sebelumnya pernah terjadi kejadian serupa, seperti Kilang Balikpapan pada Maret 2022 lalu. Terhitung 2 tahun terakhir ini, kata dia, sudah 5 fasilitas migas milik Pertamina mengalami insiden kebakaran.""Menyayangkan terjadinya musibah kebakaran ini, apalagi insiden kebakaran bukan kali pertama di fasilitas migas, ini terus berulang. Pertamina harus benahi sistem pengamanan dan SOP yang ada,"" kata dia.Rofik mengharapkan adanya pembenahan menyeluruh terhadap sistem pengamanan serta SOP yang ada dalam lingkup Pertamina untuk mengamankan baik fasilitas migas, pekerja, maupun masyarakat sekitar. Menurutnya pembenahan diperlukan agar Pertamina tidak abai terhadap aspek pengamanan fasilitas migas.""Investigasi menyeluruh dan tuntas, serta meminta komitmen Pertamina memperbaiki sistem keamanan kilang minyak maupun depo BBM, seringnya kebakaran terjadi mengindikasikan Pertamina abai terhadap pengamanan kilang,"" ujarnya.Pilihan Editor:Profil Depo Pertamina Plumpang, Pemasok 20 Persen Pasokan BBM di Seluruh Indonesia","Anggota Komisi VII DPR RI Rofik Hananto menyayangkan terjadinya insiden kebakaran di Depo Plumpang, Jakarta Utara, pada Jumat, 3 Maret 2023. Rofik menegaskan Pertamina harus memastikan pasokan bahan bakar minyak tetap aman meski ada insiden tersebut. Depo Plumpang merupak",2123,272


In [11]:
df = df.drop(['content_length' , 'summary_length'], axis=1)
df = df.reset_index(drop=True)

In [12]:
# https://www.kaggle.com/arthurtok/spooky-nlp-and-topic-modelling-tutorial

# Split the content column into batches (e.g., 5000 rows per batch)
batch_size = 100
num_batches = len(df) // batch_size + 1
batches = [df['content'].iloc[i*batch_size:(i+1)*batch_size] for i in range(num_batches)]

# Initialize an empty dictionary to store word counts
all_words_counts = {}

# Iterate over each batch and compute value_counts
for i, batch in enumerate(batches):
    batch_counts = batch.str.split(expand=True).unstack().value_counts()
    for word, count in batch_counts.items():
        all_words_counts[word] = all_words_counts.get(word, 0) + count

# Convert the dictionary to a DataFrame
all_words_df = pd.DataFrame(list(all_words_counts.items()), columns=['Word', 'Count'])

# Sort the DataFrame by count in descending order
all_words_df = all_words_df.sort_values(by='Count', ascending=False)

# Select the top 50 words
top_words_df = all_words_df.head(50)

# Plot the top 50 word frequencies
import plotly.graph_objs as go

data = [go.Bar(
            x = top_words_df['Word'],
            y = top_words_df['Count'],
            marker= dict(colorscale='Jet',
                         color = top_words_df['Count']
                        ),
            text='Word counts'
    )]

layout = go.Layout(
    title='Top 50 Word frequencies in the dataset'
)

fig = go.Figure(data=data, layout=layout)

fig.show()

In [13]:
# wc = WordCloud(width=900, height=600)

# wc.generate(','.join(df['summary']))
# plt.figure(figsize=(18,13))
# plt.imshow(wc)
# plt.axis('off')
# plt.title('summary word cloud', fontdict={'fontsize': 20})

# wc.generate(','.join(df['content']))
# plt.figure(figsize=(18,13))
# plt.imshow(wc)
# plt.axis('off')
# plt.title('content word cloud', fontdict={'fontsize': 20})

# plt.show()

In [14]:
df = df.rename(columns={'content': 'input_text', 'summary': 'target_text'}).reindex(columns=['input_text', 'target_text'])
df['prefix'] = ''

train, test = train_test_split(df, test_size=0.2, random_state=42)
train, valid = train_test_split(train, test_size=0.2, random_state=42)

## Data Augmentation <a class="anchor" id="Data-Augmentation"></a>

<img src="https://github.com/makcedward/nlpaug/blob/master/res/logo_small.png?raw=true" style="height: 300px; width: 300px;  object-position: 0px;"/>

Use nlpaug to apply data augmentation.

[Document]

[Document]: https://nlpaug.readthedocs.io/en/latest/

[Github]

[Github]: https://github.com/makcedward/nlpaug

In [15]:
aug = naw.SynonymAug(aug_src='wordnet')
augmented_text = aug.augment(list(train['input_text'].head(1)))
print("Original:")
print(','.join(train.head(1)['input_text'].values))
print()
print("Augmented Text:")
print(','.join(augmented_text))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Original:
german : Die Blindschleiche (genauer: Westliche Blindschleiche) (Anguis fragilis) ist eine Echsenart innerhalb der Familie der Schleichen (Anguidae).In Mitteleuropa gehÃ¶rt sie zu den am hÃ¤ufigsten vorkommenden Reptilien.Mit ihrem beinlosen, langgestreckten KÃ¶rper gleicht sie einer Schlange und wird auch oft fÃ¼r eine solche gehalten.Dieses MissverstÃ¤ndnis spiegelt sich sogar im wissenschaftlichen Gattungsnamen wider, den ihr Carl von Linne gegeben hat (lateinisch anguis = "Schlange"; der Artname fragilis bedeutet "zerbrechlich").Wichtige Unterscheidungsmerkmale zu den Schlangen sind das leichte Abbrechen des Schwanzes sowie das fÃ¼r alle Schleichen typische Vorhandensein von beweglichen Augenlidern und Ã¤uÃeren GehÃ¶rÃ¶ffnungen, wenn auch letztere durch Schuppen verdeckt sind.Ein anderer verbreiteter Irrtum ist, dass die Blindschleiche gemÃ¤Ã der Artbezeichnung blind sei.Der deutsche Name wird aber auf das Althochdeutsche plintslicho zurÃ¼ckgefÃ¼hrt, was nach allgemeine

After applying data augmentation, we will combine them with the original train data.

In [16]:
train.head(2)

input_text  \
10247  german : Die Blindschleiche (genauer: Westliche Blindschleiche) (Anguis fragilis) ist eine Echsenart innerhalb der Familie der Schleichen (Anguidae).In Mitteleuropa gehÃ¶rt sie zu den am hÃ¤ufigsten vorkommenden Reptilien.Mit ihrem beinlosen, langgestreckten KÃ¶rper gleicht sie einer Schlange und wird auch oft fÃ¼r eine solche gehalten.Dieses MissverstÃ¤ndnis spiegelt sich sogar im wissenschaftlichen Gattungsnamen wider, den ihr Carl von Linne gegeben hat (lateinisch anguis = "Schlange"; der Artname fragilis bedeutet "zerbrechlich").Wichtige Unterscheidungsmerkmale zu den Schlangen sind das leichte Abbrechen des Schwanzes sowie das fÃ¼r alle Schleichen typische Vorhandensein von beweglichen Augenlidern und Ã¤uÃeren GehÃ¶rÃ¶ffnungen, wenn auch letztere durch Schuppen verdeckt sind.Ein anderer verbreiteter Irrtum ist, dass die Blindschleiche gemÃ¤Ã der Artbezeichnung blind sei.Der deutsche Name wird aber auf das Althochdeutsche plintslicho zurÃ¼ckgefÃ¼hrt, was nach allgemeiner Auffassung so viel wie "blendende oder glÃ¤nzende Schleiche" bedeutet und sich auf das GlÃ¤nzen der glatten Schuppenhaut sowie die typische Fortbewegung beziehen dÃ¼rfte.Andere, heute nicht oder kaum mehr gebrÃ¤uchliche Bezeichnungen lauten Haselwurm und Hartwurm.Die Blindschleiche wurde von der Deutschen Gesellschaft fÃ¼r Herpetologie und Terrarienkunde zum Reptil des Jahres 2017 gekÃ¼rt.== Merkmale=== KÃ¶rperbauBlindschleichen haben einen langgestreckten, im Querschnitt kreisrunden KÃ¶rper ohne ExtremitÃ¤ten und erreichen eine GesamtlÃ¤nge von bis zu 57,5 cm.Die meisten zu beobachtenden erwachsenen Tiere sind aber eher zwischen 40 und 45 cm lang, wobei davon bis zu 22 cm auf den Kopf-Rumpf-Abschnitt entfallen, der Rest auf den Schwanz.Der recht kleine, hohe Kopf geht unvermittelt in den Rumpf Ã¼ber.Auch der in einer hornigen Spitze endende Schwanz ist nicht vom Rumpf abgesetzt und oft etwas lÃ¤nger als dieser.Dadurch, dass die Tiere ihren Schwanz an mehreren Sollbruchstellen leicht abwerfen kÃ¶nnen, haben allerdings recht viele Exemplare keinen vollstÃ¤ndig erhaltenen Schwanz mehr.Anders als bei Echten Eidechsen wÃ¤chst nach einer Autotomie der Schwanzabschnitt auch nicht nach.Es bildet sich nur ein sehr kurzer, halbkugeliger Stumpf.In manchen Populationen hat mehr als die HÃ¤lfte der Erwachsenen keinen vollstÃ¤ndigen Schwanz mehr.FÃ¼r feldbiologische KÃ¶rpermessungen wird daher die Kopf-Rumpf-LÃ¤nge - von der Schnauzenspitze bis zur Kloake - bevorzugt.Die Kloake hat bei der Blindschleiche einen quergestellten Spalt.Die HautoberflÃ¤che besteht aus glatten, runden bis sechseckigen, sich dachziegelartig Ã¼berlagernden Hornschuppen, die ober- und unterseits des KÃ¶rpers etwa gleich geformt sind.Auch an der Bauchseite sind davon mehrere LÃ¤ngsreihen vorhanden, und die Schuppen sind dort nur geringfÃ¼gig kleiner als auf der RÃ¼ckenseite.In der Rumpfmitte umfasst eine Querreihe 24 oder 26 Schuppen.Insgesamt weist der Rumpf 125 bis 150 Schuppenquerreihen und der Schwanz noch einmal 130 bis 160 Reihen auf.Unter den Schuppen befinden sich KnochenplÃ¤ttchen (Osteoderme), wodurch sich Blindschleichen viel steifer und plumper kriechend fortbewegen als Schlangen.Die Kopfbeschuppung Ã¤hnelt der von Eidechsen; die den Kopf nach hinten begrenzenden Pileus-Schilde sind relativ groÃ.Die OhrÃ¶ffnungen sind allerdings meistens ganz unter den Schuppen verborgen.Die relativ kleinen Augen haben bewegliche, verschlieÃbare Lider (bei Schlangen sind diese verwachsen) und runde Pupillen.Die eher kurze Zunge ist breit-zweilappig und lÃ¤uft nicht in feine Spitzen aus.Zum ZÃ¼ngeln, also zur Aufnahme von Geruchsstoffen, mÃ¼ssen Blindschleichen das Maul etwas Ã¶ffnen, da sie keine OberlippenlÃ¼cke wie die Schlangen haben.Die spitzen, teilweise recht lose sitzenden ZÃ¤hnchen sind nach hinten gekrÃ¼mmt; im Zwischenkiefer befinden sich davon 7 bis 9, im Oberkiefer 10 bis 12, im Unterkiefer 14 bis 16.In fortgeschrittenem Alter haben Blindschleichen einen Teil der Z

In [17]:
pip show nltk

Name: nltk
Version: 3.5
Summary: Natural Language Toolkit
Home-page: http://nltk.org/
Author: Steven Bird
Author-email: stevenbird1@gmail.com
License: Apache License, Version 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: click, joblib, regex, tqdm
Required-by: textblob


In [18]:
import nltk

In [19]:
  # train = pd.concat([
  #   train,
  #   pd.DataFrame({'input_text': naw.SynonymAug(aug_src='wordnet').augment(list(train['input_text'])),
  #                 'target_text': list(train['target_text']),
  #                 'prefix': ''}),
  #                 ])


def random_insertion(text, n=3):
    words = text.split()
    for _ in range(n):
        idx = random.randint(0, len(words) - 1)
        words.insert(idx, random.choice(words))
    return ' '.join(words)

trian = pd.concat([
    train,
    pd.DataFrame({'input_text': train['input_text'].apply(random_insertion),
                  'target_text': list(train['target_text']),
                  'prefix': ''}),
])

### Training

In [20]:
train_params = {
    'max_seq_length': 512,
    'max_length': 128,
    'train_batch_size': 8,
    'eval_batch_size': 8,
    'num_train_epochs': 1,
    'evaluate_during_training': True,
    'evaluate_during_training_steps': 10000,
    'use_multiprocessing': False,
    'fp16': True,
    'save_steps': -1,
    'save_eval_checkpoints': False,
    'save_model_every_epoch': False,
    'no_cache': True,
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'preprocess_inputs': False,
    'num_return_sequences': 1
}

model = T5Model('t5', 't5-small', args=train_params, use_cuda=True)
model.train_model(train, eval_data=valid)
gc.collect()

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

...

Using Adafactor for T5


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/987 [00:00<?, ?it/s]

  0%|          | 0/1974 [00:00<?, ?it/s]

41966

### Predict

In [21]:
pred_params = {
        'max_seq_length': 512,
        'use_multiprocessed_decoding': False
        }

model = T5Model('t5', 'outputs/best_model', args=pred_params, use_cuda=True)
pred = model.predict(list(test['input_text']))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Generating outputs:   0%|          | 0/309 [00:00<?, ?it/s]

In [22]:
random.sample(pred, 5)

['Menteri Koordinator Bidang Perekonomian Airlangga Hartarto mendorong kawasan ASEAN jadi pusat pertumbuhan ekonomi dunia dengan PDB mencapai US$ 3,36 triliun pada 2021. Perekonomian Indonesia di tahun 2022 mampu tumbuh solid di angka 5,3 persen year on year (yoy) dan diproyeksikan tetap memberi performa ser',
 'Die Groen Seen sind eine Gruppe von S14den im Nordamerika.Der grte Teil des Seens liegt in der kanadischen Grenze.Der grte Teil des Seens ist der Niagaraflle.Der groe Teil des Seens liegt in der kanadischen Grenze.Der groe Teil des Seens ist der kanadische Teil des Seen.Der groe Teil des Gebietes ist ein groes Gebiet.D',
 'Bank Negara Indonesia (Persero) Tbk. atau BNI berpeluang tumbuh semakin positif seiring dengan peningkatan aktivitas bisnis di dalam Keketuaan ASEAN 2023. BNI dinilai menghubungkan perdagangan sekaligus investasi Indonesia dengan negara-negara di Asia Tenggara. BNI juga terfokus pada bisnis global, akan mendapat',
 'Bursa Efek Indonesia melaporkan 14 emiten t

# Using Sumval for evaluation metrics

In [23]:
rouge = RougeCalculator(stopwords=True, lang="en")

def rouge_calc(preds, targets):
    rouge_1 = [rouge.rouge_n(summary=preds[i],references=targets[i],n=1) for i in range(len(preds))]
    rouge_2 = [rouge.rouge_n(summary=preds[i],references=targets[i],n=2) for i in range(len(preds))]
    rouge_l = [rouge.rouge_l(summary=preds[i],references=targets[i]) for i in range(len(preds))]

    return {"Rouge_1": np.array(rouge_1).mean(),
            "Rouge_2": np.array(rouge_2).mean(),
            "Rouge_L": np.array(rouge_l).mean()}

In [24]:
rouge_calc(pred, list(test['target_text']))

{'Rouge_1': 0.34761777956925666,
 'Rouge_2': 0.19102605300212888,
 'Rouge_L': 0.27875282187887096}